In [1]:
# [Cell 1] 기본 설정
import os
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# .env 로드 (API KEY 등)
load_dotenv()

# 기존 프로젝트의 설정을 가져옵니다.
# 만약 경로 문제가 생기면 sys.path.append를 사용하거나, 아래 LLM 정의를 직접 해주세요.
from src.config import LLM, DB_INSTANCE 

print("✅ 환경 설정 완료. LLM 및 VectorStore 로드됨.")

/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.0) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


FAISS 인덱스를 로드합니다...
FAISS 인덱스 로드가 완료되었습니다.
✅ 환경 설정 완료. LLM 및 VectorStore 로드됨.


In [2]:
# [Cell 2] 지역명 추출 테스트

region_prompt = PromptTemplate.from_template("""
사용자의 검색어: "{query}"
현재 여행 목적지: "{destination}"

이 검색어가 가리키는 정확한 행정구역(City, District)을 추출해.
예: "해운대 맛집" -> "부산광역시 해운대구", "문경새재" -> "경상북도 문경시"
목적지와 검색어가 상충되면 목적지를 우선해.
답변은 오직 지역명만 출력해.
""")

region_chain = region_prompt | LLM | StrOutputParser()

# 테스트 케이스
test_queries = [
    ("부산 해운대 맛집", "부산"),
    ("광안리 카페", "부산"),
    ("문경새재", "문경"),
    ("바닷가 근처 횟집", "여수") # 목적지가 여수일 때
]

print("--- [지역명 추출 테스트] ---")
for query, dest in test_queries:
    result = await region_chain.ainvoke({"query": query, "destination": dest})
    print(f"Q: '{query}' (in {dest}) -> A: {result.strip()}")

--- [지역명 추출 테스트] ---
Q: '부산 해운대 맛집' (in 부산) -> A: 부산광역시 해운대구
Q: '광안리 카페' (in 부산) -> A: 부산광역시 수영구
Q: '문경새재' (in 문경) -> A: 경상북도 문경시
Q: '바닷가 근처 횟집' (in 여수) -> A: 여수시


In [3]:
# [Cell 3] 검색 및 필터링 로직 시뮬레이션

async def test_search_logic(query, target_region):
    print(f"\n🔍 검색어: {query} / 타겟 지역: {target_region}")
    
    # 1. 넓게 검색 (k=15)
    docs = await DB_INSTANCE.asimilarity_search(query, k=15)
    print(f"👉 1차 검색 결과: {len(docs)}개 발견")
    
    # 2. 파이썬 레벨 필터링 (지역명 포함 여부 확인)
    filtered_docs = []
    
    # 타겟 지역명을 '시/군/구' 단위로 쪼개서 비교 (간이 로직)
    region_keywords = target_region.split() 
    
    for doc in docs:
        address = doc.metadata.get('address', '') # 메타데이터에 주소가 있다고 가정
        name = doc.metadata.get('name', '')
        
        # 주소에 타겟 지역 키워드가 포함되어 있는지 확인
        # (예: '부산광역시 해운대구' -> '부산' 또는 '해운대구'가 주소에 있어야 함)
        if any(keyword in address for keyword in region_keywords):
            filtered_docs.append(f"[{name}] {address}")
        else:
            # 탈락한 친구들 로그 찍어보기 (디버깅용)
            # print(f"   ❌ 탈락: [{name}] {address}") 
            pass
            
    print(f"👉 필터링 후 결과: {len(filtered_docs)}개 남음")
    for item in filtered_docs[:5]: # 상위 5개만 출력
        print(f"   ✅ {item}")

# 실행
await test_search_logic("분위기 좋은 카페", "부산광역시 해운대구")


🔍 검색어: 분위기 좋은 카페 / 타겟 지역: 부산광역시 해운대구


I0000 00:00:1764998312.324019 7467618 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


👉 1차 검색 결과: 15개 발견
👉 필터링 후 결과: 0개 남음


In [4]:
# [Cell 4] 개인화 설명 생성 테스트

desc_prompt = PromptTemplate.from_template("""
사용자 정보: {user_info}
장소 이름: {place_name}
장소 특징/리뷰: {place_data}

위 정보를 바탕으로 이 장소가 이 사용자(모임 유형, 선호도 등)에게 왜 좋은지 매력적인 1~2줄의 추천사를 작성해. 
반드시 한국어로 작성하고, 문장 끝은 '해요', '좋아요' 등으로 자연스럽게.
""")
desc_chain = desc_prompt | LLM | StrOutputParser()

# 더미 데이터
dummy_user_info = "20대 대학생 친구끼리 여행, 사진 찍기 좋아함, 인스타 감성 선호"
dummy_place_name = "웨이브온 커피"
dummy_place_data = "바다가 바로 보이는 오션뷰 카페, 건축상 수상, 주말에 웨이팅 있음, 디저트 맛있음"

print("--- [개인화 추천사 생성] ---")
description = await desc_chain.ainvoke({
    "user_info": dummy_user_info,
    "place_name": dummy_place_name,
    "place_data": dummy_place_data
})

print(f"📍 장소: {dummy_place_name}")
print(f"🗣️ 추천사: {description}")

--- [개인화 추천사 생성] ---
📍 장소: 웨이브온 커피
🗣️ 추천사: 바다가 한눈에 보이는 건축상 수상 오션뷰와 맛있는 디저트까지, 웨이브온 커피는 친구들과 인스타 감성 인생샷을 남기기 정말 좋은 곳이에요.


In [5]:
# [Cell 5] 타임라인 데이터 가공 로직

import json

def mock_timeline_parser(itinerary_list):
    """
    경로 API 결과를 받았다고 가정하고, 
    텍스트 요약에 쓸 포맷(버스 번호 등)을 추가하는 로직 검증
    """
    processed_list = []
    
    for item in itinerary_list:
        # type이 'move'인 경우 (가상의 시나리오)
        if item.get('type') == 'move':
            # API에서 이런 데이터를 받았다고 가정
            mock_api_transport_mode = "BUS" 
            mock_api_bus_no = "1003번"
            mock_duration = 25 # 분
            
            # 파싱 로직 적용
            detail_text = ""
            if mock_api_transport_mode == "BUS":
                detail_text = f"{mock_api_bus_no} 버스"
            elif mock_api_transport_mode == "SUBWAY":
                detail_text = "지하철 2호선"
            
            # duration_text 필드 생성 (이게 Streamlit에 뜰 예정)
            item['duration_text'] = f"약 {mock_duration}분 ({detail_text})"
            
        processed_list.append(item)
    
    return processed_list

# 테스트 데이터
raw_itinerary = [
    {"type": "spot", "name": "부산역"},
    {"type": "move", "from": "부산역", "to": "해운대"},
    {"type": "spot", "name": "해운대 해수욕장"}
]

final_json = mock_timeline_parser(raw_itinerary)
print(json.dumps(final_json, indent=2, ensure_ascii=False))

[
  {
    "type": "spot",
    "name": "부산역"
  },
  {
    "type": "move",
    "from": "부산역",
    "to": "해운대",
    "duration_text": "약 25분 (1003번 버스)"
  },
  {
    "type": "spot",
    "name": "해운대 해수욕장"
  }
]


In [1]:
# [Cell 1] 환경 설정 및 SmartScheduler 로드
import sys
import os
import json
import asyncio
from dotenv import load_dotenv

# .env 파일 로드 (API KEY)
load_dotenv()

# 경로 설정 (필요시)
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

from src.config import GMAPS_CLIENT
from src.scheduler.smart_scheduler import SmartScheduler

# API 키 확인
if GMAPS_CLIENT:
    print("✅ Google Maps Client 로드 성공")
else:
    print("❌ Google Maps Client 로드 실패 (API 키 확인 필요)")

# SmartScheduler 확인
try:
    scheduler = SmartScheduler(start_time_str="10:00")
    print("✅ SmartScheduler 인스턴스 생성 성공")
except Exception as e:
    print(f"❌ SmartScheduler 생성 실패: {e}")

/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.0) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FAISS 인덱스를 로드합니다...
FAISS 인덱스 로드가 완료되었습니다.
✅ Google Maps Client 로드 성공
✅ SmartScheduler 인스턴스 생성 성공


In [3]:
# [Cell 2] 실제 경로 데이터 조회 및 파싱 테스트

async def test_real_timeline_parsing():
    print("--- 🚀 실제 Google Maps API 호출 시작 ---")
    
    # 1. 테스트 데이터 (부산 실제 경로)
    # 이름만 주면 SmartScheduler가 내부적으로 좌표를 따거나 검색해서 처리합니다.
    test_places = [
        {"name": "부산역", "day": 1, "type": "activity"},
        {"name": "해운대 해수욕장", "day": 1, "type": "activity"},
        {"name": "광안리 해수욕장", "day": 1, "type": "activity"} 
    ]
    
    print(f"📍 경로: {[p['name'] for p in test_places]}")

    # 2. SmartScheduler 실행 (API 호출 발생)
    # plan_day 내부에서 길찾기 API를 호출하고 결과를 리턴합니다.
    try:
        raw_timeline = await scheduler.plan_day(test_places)
        print("✅ API 응답 수신 완료!\n")
    except Exception as e:
        print(f"❌ API 호출 중 에러 발생: {e}")
        return

    # 3. 파싱 로직 적용 (src/tools.py에 들어갈 로직 검증)
    print("--- 🛠️ 파싱 로직 적용 결과 ---")
    
    final_timeline = []
    
    for item in raw_timeline:
        # SmartScheduler가 이미 'transport_detail'(버스번호 등)을 채워서 준다고 가정
        # 만약 여기서 'transport_detail'이 비어있다면 Scheduler 내부 로직을 손봐야 함
        
        if item.get('type') == 'move':
            mode = item.get('transport_mode', '이동')
            duration_min = item.get('duration_min', 0)
            
            # 여기가 핵심: 실제 API에서 가져온 버스 번호나 지하철 호선
            detail = item.get('transport_detail', '') 
            
            # UI에 보여줄 최종 텍스트 생성
            if detail:
                item['duration_text'] = f"약 {duration_min}분 ({detail})"
            else:
                item['duration_text'] = f"약 {duration_min}분 ({mode})"
            
            print(f"   ⬇️ [이동] {item['duration_text']}")
            print(f"       (Raw Data: mode={mode}, detail='{detail}')")
            
        else:
            print(f"   📍 [장소] {item.get('name')} ({item.get('start_time')} ~ {item.get('end_time')})")
        
        final_timeline.append(item)

    print("\n--- 📄 최종 JSON 구조 (Top 3 Items) ---")
    print(json.dumps(final_timeline[:3], indent=2, ensure_ascii=False))

# 비동기 실행
await test_real_timeline_parsing()

--- 🚀 실제 Google Maps API 호출 시작 ---
📍 경로: ['부산역', '해운대 해수욕장', '광안리 해수욕장']
DEBUG: 🚗 경로 계산: '부산역' -> '해운대 해수욕장'
DEBUG: 🚗 경로 계산: '해운대 해수욕장' -> '광안리 해수욕장'
✅ API 응답 수신 완료!

--- 🛠️ 파싱 로직 적용 결과 ---
   📍 [장소] 부산역 (None ~ None)
   ⬇️ [이동] 약 54분 ([버스] 1003)
       (Raw Data: mode=transit, detail='[버스] 1003')
   📍 [장소] 해운대 해수욕장 (None ~ None)
   ⬇️ [이동] 약 41분 ([버스] 1003)
       (Raw Data: mode=transit, detail='[버스] 1003')
   📍 [장소] 광안리 해수욕장 (None ~ None)

--- 📄 최종 JSON 구조 (Top 3 Items) ---
[
  {
    "type": "activity",
    "name": "부산역",
    "category": "activity",
    "start": "10:00",
    "end": "11:30",
    "duration_minutes": 90,
    "description": ""
  },
  {
    "type": "move",
    "from": "부산역",
    "to": "해운대 해수욕장",
    "start": "11:30",
    "end": "12:24",
    "duration_min": 54,
    "transport_mode": "transit",
    "transport_detail": "[버스] 1003",
    "duration_text_raw": "54분",
    "duration_text": "약 54분 ([버스] 1003)"
  },
  {
    "type": "activity",
    "name": "해운대 해수욕장",
    "category

In [1]:
# [Cell] 검색 엔진 성능 검증 (Real Integration Test)

import asyncio
import json
import pandas as pd
from tqdm.asyncio import tqdm

# 🚨 실제 소스코드의 함수를 가져옵니다. (흉내 X)
from src.tools import find_and_select_best_place, resolve_admin_region
from src.config import DB_INSTANCE

# ==========================================
# 1. 테스트 케이스 정의 (1000개 확장 가능)
# ==========================================
# (입력 쿼리, 목적지) -> (예상되는 변환 지역명, 결과 주소에 포함되어야 할 핵심 단어)
TEST_CASES = [
    # [Case A] 비공식 지명 (POI) 테스트
    {"query": "오션뷰 카페", "dest": "부산", "anchor": "광안리", "expect_region": "수영구", "desc": "광안리 -> 수영구 변환"},
    {"query": "맛집", "dest": "부산", "anchor": "해리단길", "expect_region": "해운대구", "desc": "해리단길 -> 해운대구 변환"},
    {"query": "길거리 음식", "dest": "전주", "anchor": "한옥마을", "expect_region": "완산구", "desc": "한옥마을 -> 완산구 변환"},
    
    # [Case B] 공식 행정구역 테스트
    {"query": "파스타", "dest": "서울 강남구", "anchor": "", "expect_region": "강남구", "desc": "서울 강남구 유지"},
    {"query": "국밥", "dest": "부산", "anchor": "서면", "expect_region": "부산진구", "desc": "서면 -> 부산진구 변환"},
    
    # [Case C] 모호한 검색어
    {"query": "조용한 카페", "dest": "제주", "anchor": "애월", "expect_region": "제주시", "desc": "애월 -> 제주시 변환"},
]

# ==========================================
# 2. 검증 엔진
# ==========================================
async def run_test(case):
    log = {
        "Case": case["desc"],
        "Input": f"{case['dest']} {case['anchor']} {case['query']}",
        "Expected": case["expect_region"],
        "Admin_Convert": "FAIL",   # resolve_admin_region 결과
        "Search_Filter": "FAIL",   # find_and_select_best_place 결과
        "Final_Place": "",
        "Final_Address": ""
    }
    
    # -------------------------------------------------
    # Step 1: 행정구역 변환 로직 검증 (resolve_admin_region 호출)
    # -------------------------------------------------
    # 사용자가 '광안리'라고 했을 때, 내부 함수가 실제로 '수영구'를 리턴하는지 확인
    search_input = case["anchor"] if case["anchor"] else case["query"]
    
    try:
        # 실제 함수 호출
        resolved_region = await resolve_admin_region(search_input, case["dest"])
        
        if case["expect_region"] in resolved_region:
            log["Admin_Convert"] = f"✅ PASS ({resolved_region})"
        else:
            log["Admin_Convert"] = f"❌ FAIL (결과: {resolved_region})"
    except Exception as e:
        log["Admin_Convert"] = f"❌ ERROR ({str(e)})"

    # -------------------------------------------------
    # Step 2: 통합 검색 및 필터링 검증 (find_and_select_best_place 호출)
    # -------------------------------------------------
    # 실제로 DB를 검색하고 필터링해서 최종 1개를 뽑았을 때, 그 주소가 맞는지 확인
    try:
        # 실제 도구 호출
        result_json = await find_and_select_best_place.ainvoke({
            "query": case["query"],
            "destination": case["dest"],
            "anchor": case["anchor"],
            "exclude_places": [],
            "user_info": "검증용 봇" # 설명 생성용 더미 데이터
        })
        
        # 결과 파싱
        result = json.loads(result_json)
        place_name = result.get("name", "없음")
        address = result.get("address", "")
        
        log["Final_Place"] = place_name
        log["Final_Address"] = address
        
        # 검증: 최종 결과물의 주소에 '수영구'가 들어있는가?
        if case["expect_region"] in address:
            log["Search_Filter"] = "✅ PASS"
        else:
            # 엉뚱한 지역(예: 해운대구)이 나왔거나 검색 실패
            log["Search_Filter"] = "❌ FAIL (주소 불일치)"
            
    except Exception as e:
        log["Search_Filter"] = f"❌ ERROR ({str(e)})"
        
    return log

# ==========================================
# 3. 실행 및 리포트
# ==========================================
results = []
print("🚀 검색 엔진 정합성 테스트 시작...\n")

for case in tqdm(TEST_CASES):
    res = await run_test(case)
    results.append(res)

# 데이터프레임으로 깔끔하게 출력
df = pd.DataFrame(results)

# 스타일링 (터미널 출력용)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 50)

print("\n📊 [테스트 결과 리포트]")
display(df[["Case", "Expected", "Admin_Convert", "Search_Filter", "Final_Place", "Final_Address"]])

# 실패한 케이스만 보기
failures = df[df["Search_Filter"].str.contains("FAIL")]
if not failures.empty:
    print("\n🚨 [주의] 필터링 실패 케이스:")
    display(failures)
else:
    print("\n✨ 모든 테스트 케이스가 성공했습니다! 로직이 완벽합니다.")

/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/Caskroom/miniforge/base/envs/env10/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.0) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


FAISS 인덱스를 로드합니다...
FAISS 인덱스 로드가 완료되었습니다.
🚀 검색 엔진 정합성 테스트 시작...



  0%|          | 0/6 [00:00<?, ?it/s]

DEBUG: 🗺️ 행정구역 식별 시도: '부산 광안리'
DEBUG: ✅ 변환 성공: '광안리' -> '부산광역시 수영구'

--- [DEBUG] find_and_select_best_place 호출 ---
QUERY: 오션뷰 카페 / ANCHOR: 광안리 / DEST: 부산
DEBUG: ⚓️ 앵커 기반 지역 변환 시도: '광안리'
DEBUG: 🗺️ 행정구역 식별 시도: '부산 광안리'
DEBUG: ✅ 변환 성공: '광안리' -> '부산광역시 수영구'
DEBUG: 🎯 확정 타겟 지역: '부산광역시 수영구'
DEBUG: ⚙️ 필터 키워드: ['부산', '수영']


 17%|█▋        | 1/6 [00:09<00:49,  9.83s/it]

...

100%|██████████| 6/6 [00:58<00:00,  9.78s/it]

✅ 최종 추천: 마음에온

📊 [테스트 결과 리포트]


,Case,Expected,Admin_Convert,Search_Filter,Final_Place,Final_Address
0,광안리 -> 수영구 변환,수영구,✅ PASS (부산광역시 수영구),✅ PASS,프론트커피 광안리,부산광역시 수영구
1,해리단길 -> 해운대구 변환,해운대구,✅ PASS (부산광역시 해운대구),✅ PASS,해목,부산광역시 해운대구
2,한옥마을 -> 완산구 변환,완산구,✅ PASS (전북특별자치도 완산구),❌ FAIL (주소 불일치),민속전주식당,전북특별자치도 완주군
3,서울 강남구 유지,강남구,✅ PASS (서울특별시 강남구),✅ PASS,어거스트힐 강남점,서울특별시 강남구
4,서면 -> 부산진구 변환,부산진구,❌ FAIL (결과: 부산광역시),❌ FAIL (주소 불일치),영진돼지국밥 경성부경점,부산광역시 남구
5,애월 -> 제주시 변환,제주시,✅ PASS (제주특별자치도 제주시),✅ PASS,마음에온,제주특별자치도 제주시



🚨 [주의] 필터링 실패 케이스:


,Case,Input,Expected,Admin_Convert,Search_Filter,Final_Place,Final_Address
2,한옥마을 -> 완산구 변환,전주 한옥마을 길거리 음식,완산구,✅ PASS (전북특별자치도 완산구),❌ FAIL (주소 불일치),민속전주식당,전북특별자치도 완주군
4,서면 -> 부산진구 변환,부산 서면 국밥,부산진구,❌ FAIL (결과: 부산광역시),❌ FAIL (주소 불일치),영진돼지국밥 경성부경점,부산광역시 남구
